# MediaTek Genio AI 開發指南-2：TensorFlow Lite 模型效能測試

## 🎯 關於這個筆記本
這是一個完整的教學範例，教您如何在 MediaTek Genio SoC（系統晶片）上使用 NeuronRT 運行 AI 模型，並測量其效能。

## 📚 背景知識：
- **TensorFlow Lite**：Google 開發的輕量級 AI 推論框架，專為手機、嵌入式裝置設計，支援`.tflite`格式的AI模型。
- **NeuronRT**：MediaTek 公司開發的神經網路推論引擎，可以加速推論框架背景的 AI計算工作
- **MediaTek Genio SoC**：聯發科的 AI 處理器，內建與NeuronRT相容的DLA及VPU

## 📋 您將學會：
- 如何透過TFLite Runtime載入和配置 AI 模型
- 如何使用 NeuronRT於DLA 或 VPU 上進行 AI 推論加速
- 如何測量 AI 模型的執行速度和記憶體使用量
- 如何將這些技術整合到您的應用程式中

## 1. 環境設置

我們需要切換到專案的根目錄，這樣程式才能找到模型檔案和其他必要的資源。

In [ ]:
% cd ..

## 2. 匯入必要程式庫

載入Tensorflow及 Python 程式庫來處理 AI 推論的工作流程。

In [ ]:
import time, warnings, os
import numpy as np
from tqdm import tqdm
from utils.neuronpilot.neuronrt import Interpreter
warnings.simplefilter('ignore')

print(f"NumPy 版本: {np.__version__}")

## 3. 設定測試參數

根據您的 Genio 平台及測試需求，設定要使用的加速器、AI模型以及測試參數。

### 📋 參數說明：
- **device**：加速器類型，支援多種選項：
  - `mdla3.0`：適用於 Genio 510/700，使用 MDLA v3.0
  - `mdla2.0`：適用於 Genio 1200，使用 MDLA v2.0  
  - `vpu`：使用 VPU 加速器進行推理
- **tflite_path**：TensorFlow Lite 模型檔案路徑（用於取得模型資訊）
- **dla_path**：編譯好的 DLA 模型檔案路徑（用於實際推理）

> **🔧 如何取得 DLA 模型：**
> 
> 您可以透過工研院提供的 [NeuronPilot Porting Platform](https://neuronpilot-ai-porting-platform.azurewebsites.net/) 線上平台，將您的 TensorFlow Lite 模型轉換為 DLA 格式：<br>**[NeuronPilot Platform]** > **[Upload Prebuilt Model]** > **[選擇 TFLite 模型]** > **[Upload and Verify Model]** > **[選擇開發板與 Device]**> **[Download DLA]**

In [ ]:
device = 'mdla3.0'
tflite_path = "models/yolov8n_float32.tflite"
dla_path = "models/yolov8n_float32_mdla3.dla"

## 4. 建立 NeuronRT Interpreter

使用我們自訂的 `utils.neuronpilot.neuronrt.Interpreter` 類別建立NeuronRT 解釋器。

In [ ]:
from utils.neuronpilot.neuronrt import Interpreter

interpreter = Interpreter(
    tflite_path=tflite_path, 
    dla_path=dla_path, 
    device=device
)

print("✅ utils.neuronpilot.neuronrt.Interpreter 建立完成")
print(f"🔧 使用加速器: {device.upper()}")

## 5. 初始化 NeuronRT 解釋器

建立 NeuronRT 解釋器並配置 AI 模型。
1. 分配模型所需的記憶體空間
2. 載入 DLA 模型到指定的加速器
3. 取得模型的輸入和輸出資訊

In [ ]:
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']
input_dtype = input_details[0]['dtype']

print("📋 模型資訊:")
print(f"   輸入形狀: {input_details[0]['shape']}")
print(f"   資料型態: {input_details[0]['dtype']}")
print(f"   輸出形狀: {output_details[0]['shape']}")
print(f"   資料型態: {output_details[0]['dtype']}")

## 6. 準備輸入資料

為 AI 模型準備測試用的輸入資料。
1. 根據模型要求的形狀產生隨機數字陣列
2. 轉換資料型態以符合模型規格
3. 將準備好的資料送入模型

In [ ]:
inputs = np.random.rand(*input_details[0]['shape']).astype(input_details[0]['dtype'])

print(f"📊 測試資料形狀: {inputs.shape}")
print(f"📊 測試資料型別: {inputs.dtype}")
print(f"📊 測試資料範圍: [{inputs.min():.3f}, {inputs.max():.3f}]")

interpreter.set_tensor(0, inputs)

## 7. 執行效能基準測試

執行多次推論來測量模型的平均效能。
1. 設定測試的執行次數
2. 記錄每次推論的執行時間
3. 計算平均效能和吞吐量

In [ ]:
iteration = 10
print(f"開始執行 {iteration} 次推論測試...")
inference_start_time = time.time()

for i in tqdm(range(iteration), desc="推論進度", unit="次"):
    interpreter.invoke()

inference_end_time = time.time()
outputs = interpreter.get_tensor(output_details[0]['index'])

## 7. 顯示測試結果

計算並顯示 AI 模型的效能指標。
1. 計算平均單次推論時間（毫秒）
2. 計算推論吞吐量（每秒處理次數）
3. 以表格格式顯示完整的效能報告

In [ ]:
total_inference_time = inference_end_time - inference_start_time
avg_inference_time_ms = total_inference_time * 1000 / iteration

print('=' * 50)
print('TensorFlow Lite 模型效能測試結果')
print('=' * 50)
print(f'單次推論時間: {avg_inference_time_ms:.2f} ms')
print(f'推論吞吐量: {1000 / avg_inference_time_ms:.2f} FPS')
print('=' * 50)

## 9. 整合到實際應用

您可以參考以下的程式範例，將ArmNN加速方案整合到您的AI工作流程中。

```python
from utils.neuronpilot.neuronrt import Interpreter

interpreter = Interpreter(
    tflite_path="<path_to your_tflite_model>", 
    dla_path="<path_to_your_dla_model>",       
    device= "<mdla3.0, mdla2.0 or vpu>"
)

interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(0, test_input)
interpreter.invoke()
output = interpreter.get_tensor(0)
```